In [2]:
import veryfi
import os
import json
import re

# Configuración de credenciales de Veryfi
client_id = 'vrfhjAs5kPIee2Yvy7F0osVV5MX6zXKFTriJyLS'
client_secret = 'NJoXZcE25MBxv97merROYW5CvYBJLmNwrGDoYEWzv970T0fyeE2pxAXlP93huX46Bf1NbgM6jGzVb3LzSrTWZYDEGcyaruHwfWAr75EfA87rcoo29NLYzWRWJdogbc1N'
username = 'juanpamon.13'
api_key = 'f604ee697fd73c0ba1bb99a536bcde0b'

client = veryfi.Client(client_id, client_secret, username, api_key)

# Directorios
input_folder = "Documents"
veryfi_output_folder = "veryfi_json"
processed_output_folder = "output"

# Crear las carpetas de salida si no existen
os.makedirs(veryfi_output_folder, exist_ok=True)
os.makedirs(processed_output_folder, exist_ok=True)

def extract_invoice_data(ocr_text):
    """
    Extrae la información clave de la factura desde el texto OCR.
    """

    invoice_data = {}

    # Vendor Name
    vendor_match = re.search(r'Please make payments to:\s*(.+)', ocr_text)
    invoice_data["vendor_name"] = vendor_match.group(1).strip() if vendor_match else "Unknown"

    # Vendor Address (Fijo)
    #vendor_address_match = re.search(r"(?:(?:Page \d+ of \d+\n)?\s*)(\S+\s*.*?)\n(?:\n|\t)Invoice Date", ocr_text, re.DOTALL | re.MULTILINE)
    vendor_address_match = re.search(r"\n([\w\s,]+ \d{5}(?:-\d{4})?)\nPO Box", ocr_text)

    if vendor_address_match:
        address = vendor_address_match.group(1).strip()
        address_parts = address.split()[-3:]  # Tomar solo las últimas dos partes (ciudad y código postal)
        invoice_data["vendor_address"] = " ".join(address_parts)
    else:
        invoice_data["vendor_address"] = "Unknown"
    # Invoice Number y Bill To Name
    invoice_match = re.search(r'Invoice No\.\n\t(\d{2}/\d{2}/\d{2})\t(\d{2}/\d{2}/\d{2})\t([\d]+)\n\n(.+)', ocr_text)
    if invoice_match:
        invoice_data["date"] = invoice_match.group(1)  # Primera fecha después de Invoice No.
        invoice_data["invoice_number"] = invoice_match.group(3)  # Número de factura
        invoice_data["bill_to_name"] = invoice_match.group(4).strip()  # Nombre después de Invoice No.
    else:
        invoice_data["date"] = "Unknown"
        invoice_data["invoice_number"] = "Unknown"
        invoice_data["bill_to_name"] = "Unknown"

    # Extraer Items
    items = []
    item_pattern = re.findall(
        r'([^\n]+?)\s+\(?(\d+)?\)?\s+([\d,]+\.\d+)\s+([\d,]+\.\d+)\s+([\d,]+\.\d+)',
        ocr_text
    )

    for item in item_pattern:
        description, sku, quantity, price, total = item
        items.append({
            "sku": sku if sku else "Unknown",
            "description": description.strip(),
            "quantity": float(quantity.replace(',', '')),
            "tax_rate": None,  # No está presente en la factura
            "price": float(price.replace(',', '')),
            "total": float(total.replace(',', ''))
        })

    invoice_data["items"] = items

    return invoice_data

# Procesar todos los archivos PDF en la carpeta Documents
for file_name in os.listdir(input_folder):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(input_folder, file_name)
        print(f"Procesando: {file_name}")
        
        # Extraer datos con Veryfi
        json_result = client.process_document(file_path)
        
        # Guardar el resultado en la carpeta veryfi_json
        veryfi_output_file = os.path.join(veryfi_output_folder, f"{file_name}.json")
        with open(veryfi_output_file, "w", encoding="utf-8") as f:
            json.dump(json_result, f, ensure_ascii=False, indent=4)
        
        print(f"Resultado guardado en: {veryfi_output_file}")
        
        # Procesar JSON y extraer datos clave
        with open(veryfi_output_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            ocr_text = data.get("ocr_text", "")
            invoice_info = extract_invoice_data(ocr_text)
        
        # Guardar el resultado procesado en la carpeta output
        processed_output_file = os.path.join(processed_output_folder, f"{file_name}.json")
        with open(processed_output_file, "w", encoding="utf-8") as f:
            json.dump(invoice_info, f, ensure_ascii=False, indent=4)
        
        print(f"Datos procesados guardados en: {processed_output_file}")

print("Procesamiento completado.")


Procesando: synth-switch_v5-14.pdf
Resultado guardado en: veryfi_json\synth-switch_v5-14.pdf.json
Datos procesados guardados en: output\synth-switch_v5-14.pdf.json
Procesando: synth-switch_v5-4.pdf


KeyboardInterrupt: 

In [19]:
import re
import json

def extract_invoice_data(ocr_text):
    """
    Extrae la información clave de la factura desde el texto OCR.
    """

    invoice_data = {}

    # Vendor Name
    vendor_match = re.search(r'Please make payments to:\s*(.+)', ocr_text)
    invoice_data["vendor_name"] = vendor_match.group(1).strip() if vendor_match else "Unknown"

    # Vendor Address (Fijo)
    vendor_address_match = re.search(
    r"(?<=\n)([A-Za-z\s,.-]+\d{5}(?:-\d{4})?\n(?:.*\d+)?)",
    ocr_text,
    re.DOTALL | re.MULTILINE
)
    invoice_data["vendor_address"] = vendor_address_match.group(1).strip() if vendor_address_match else "Unknown"

    # Invoice Number y Bill To Name
    invoice_match = re.search(r'Invoice No\.\n\t(\d{2}/\d{2}/\d{2})\t(\d{2}/\d{2}/\d{2})\t([\d]+)\n\n(.+)', ocr_text)
    if invoice_match:
        invoice_data["date"] = invoice_match.group(1)  # Primera fecha después de Invoice No.
        invoice_data["invoice_number"] = invoice_match.group(3)  # Número de factura
        invoice_data["bill_to_name"] = invoice_match.group(4).strip()  # Nombre después de Invoice No.
    else:
        invoice_data["date"] = "Unknown"
        invoice_data["invoice_number"] = "Unknown"
        invoice_data["bill_to_name"] = "Unknown"

    # Extraer Items
    items = []
    item_pattern = re.findall(
        r'([^\n]+?)\s+\(?(\d+)?\)?\s+([\d,]+\.\d+)\s+([\d,]+\.\d+)\s+([\d,]+\.\d+)',
        ocr_text
    )

    for item in item_pattern:
        description, sku, quantity, price, total = item
        items.append({
            "sku": sku if sku else "Unknown",
            "description": description.strip(),
            "quantity": float(quantity.replace(',', '')),
            "tax_rate": None,  # No está presente en la factura
            "price": float(price.replace(',', '')),
            "total": float(total.replace(',', ''))
        })

    invoice_data["items"] = items

    return invoice_data

# OCR Text de prueba
#ocr_text = '''(Pega aquí tu texto OCR)'''

# Extraer datos
invoice_info = extract_invoice_data(ocr_text)

# Guardar en JSON
with open("outputs/invoice_data.json", "w", encoding="utf-8") as f:
    json.dump(invoice_info, f, ensure_ascii=False, indent=4)

print(json.dumps(invoice_info, indent=4, ensure_ascii=False))

{
    "vendor_name": "Switch, Ltd.",
    "vendor_address": "switch\tDallas, TX 75267-4592\nPO Box 674592\t\t\t\t\t\t\t\tPage 1 of 1\n\n\tInvoice Date Due Date\tInvoice No.\n\t09/22/23\t08/27/24\t1556267\n\nToni Hackel\n6600 Kalanianaole Hwy Ste 200,\nHonolulu\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of March\nU-08805490\t\t\t\tPO-2OPC-4316\nDescription\t\t\t\tQuantity\tRate\tAmount\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCa

# Vendor Adress Exploration

In [9]:
import re

ocr_text = """Invoice\nswitch\tDallas, TX 75267-4592\nPO Box 674592\t\t\t\t\t\t\t\tPage 1 of 1\n\n\tInvoice Date Due Date\tInvoice No.\n\t09/22/23\t08/27/24\t1556267\n\nToni Hackel\n6600 Kalanianaole Hwy Ste 200,\nHonolulu\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of March\nU-08805490\t\t\t\tPO-2OPC-4316\nDescription\t\t\t\tQuantity\tRate\tAmount\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCarrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\nand Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\nTransport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21\n\n\tTotal USD\t$58,164,641.64\n\nPlease update your system with our new remittance instruction located on this invoice. Please contact accounts receivable@switch.com with any\nquestions.\n\nPlease make payments to: Switch, Ltd.\nWire/ACH Payment: PNC Bank, N.A.\tAccount No.:3802512896 ACH Routing:083009060 Wire Routing: 043000096 SWIFT:PNCCUS33\nFor questions, please call or email Accounts Receivable Department.\nPhone No.\tFax No.\t\t\tE-Mail\t\t\t\t\tWeb Site\n702.522.2940\t702.444.9546\taccountsreceivable@switch.com\t\t\twww.switch.com\nPlease remit payment for amount due. If payment submitted is different from amount due, a detailed statement of where to apply the funds must be included\nwith payment."""


# Expresión regular ajustada
vendor_address_match = re.search(
    r"\n([\w\s,]+ \d{5}(?:-\d{4})?)\nPO Box",
    ocr_text, 
    re.DOTALL | re.MULTILINE
)
invoice_data = {}
if vendor_address_match:
    address = vendor_address_match.group(1).strip()
    address_parts = address.split()[-3:]  # Tomar solo las últimas dos partes (ciudad y código postal)
    invoice_data["vendor_address"] = " ".join(address_parts)
else:
    invoice_data["vendor_address"] = "Unknown"

print(invoice_data["vendor_address"])

Dallas, TX 75267-4592


# Item separation exploration

In [9]:
import re

# Texto extraído del OCR
ocr_text_2 = """
Invoice\nswitch\tDallas, TX 75267-4592\nPO Box 674592\t\t\t\t\t\t\t\tPage 1 of 1\n\n\tInvoice Date Due Date\tInvoice No.\n\t09/22/23\t08/27/24\t1556267\n\nToni Hackel\n6600 Kalanianaole Hwy Ste 200,\nHonolulu\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of March\nU-08805490\t\t\t\tPO-2OPC-4316\nDescription\t\t\t\tQuantity\tRate\tAmount\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCarrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\nand Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\nTransport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21\n\n\tTotal USD\t$58,164,641.64\n"""

# Paso 1: Unir líneas partidas correctamente
ocr_lines = ocr_text_2.split("\n")
merged_lines = []
buffer = ""

for line in ocr_lines:
    if re.search(r"\t\d{1,3}(,\d{3})*\.\d{2}\t-?\d{1,3}(,\d{3})*\.\d{2}\t-?\d{1,3}(,\d{3})*\.\d{2}", line):
        if buffer:
            merged_lines.append(buffer.strip())  # Guardar buffer previo si hay datos acumulados
        buffer = line
    else:
        buffer += " " + line.strip()  # Concatenar líneas en la misma descripción

if buffer:
    merged_lines.append(buffer.strip())

# Paso 2: Extraer los datos con regex
pattern = re.compile(r"(.+?)\t+(-?[\d,]+\.?\d*)\t+(-?[\d,]+\.?\d*)\t+(-?[\d,]+\.?\d*)")
items = pattern.findall("\n".join(merged_lines))

# Imprimir resultados
print("Items Vendidos:")
for item in items:
    description, quantity, rate, amount = item
    print(f"Descripción: {description.strip()}\nCantidad: {quantity}\nRate: {rate}\nAmount: {amount}\n")


Items Vendidos:
Descripción: Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber
Cantidad: 2,912.98
Rate: 934.09
Amount: 2,720,985.49

Descripción: Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and
Cantidad: 3,500.87
Rate: 6,229.33
Amount: 21,808,074.52

Descripción: Special Partnership Discount (03/2023)
Cantidad: 8.00
Rate: -561.00
Amount: -4,488.00

Descripción: Item Discount: Special Partnership Discount (11/2023)
Cantidad: 8.00
Rate: -296.00
Amount: -2,368.00

Descripción: Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)
Cantidad: 5,607.34
Rate: 656.07
Amount: 3,678,807.55

Descripción: Carrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765
Cantidad: 3,949.46
Rate: 7,128.95
Amount: 28,155,502.87

Descripción: Transport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)
Cantidad: 942.71
Rate: 1,918.01
Amount: 1,808,127.21



## try 2

In [5]:
import re

# Texto extraído del OCR

# Paso 1: Unir líneas partidas correctamente
ocr_lines = ocr_text.split("\n")
merged_lines = []
buffer = ""

for line in ocr_lines:
    if re.search(r"\t\d{1,3}(,\d{3})*\.\d{2}\t\d{1,3}(,\d{3})*\.\d{2}\t\d{1,3}(,\d{3})*\.\d{2}", line):
        if buffer:
            merged_lines.append(buffer.strip())
        buffer = line
    else:
        buffer += " " + line.strip()

if buffer:
    merged_lines.append(buffer.strip())

# Paso 2: Extraer los datos con regex
pattern = re.compile(r"(.+?)\t+([\d,]+\.?\d*)\t+([\d,]+\.?\d*)\t+([\d,]+\.?\d*)")
items = pattern.findall("\n".join(merged_lines))

# Paso 3: Imprimir resultados corregidos
print("Items Vendidos:")
for item in items:
    description, quantity, rate, amount = item
    print(f"Descripción: {description.strip()}\nCantidad: {quantity}\nRate: {rate}\nAmount: {amount}\n")


Items Vendidos:
Descripción: Installation of Cross Connect | 395 Gbps Fiber to AE9qC3
Cantidad: 579.10
Rate: 1,750.30
Amount: 1,013,598.73

Descripción: Transport | 71 Gbps Fiber to 04pBZ (02FDIWHG) (07/2023)
Cantidad: 5,519.81
Rate: 5,201.91
Amount: 28,713,554.84

Descripción: Transport | Switch Fiber Pair (Intra-campus) | Pairs (0902986589) (03/2023 | 10 Gbps Fiber
Cantidad: 8,360.15
Rate: 930.70
Amount: 7,780,791.61

Descripción: Transport | Switch Fiber Pair (Intra-campus) | Pairs (5659868303) (01/2024|10 Gbps Fiber
Cantidad: 2,983.62
Rate: 504.07
Amount: 1,503,953.33

Descripción: Transport | 773 Gbps Fiber to 6YDZ35e (BF3EE94B) (06/2023)
Cantidad: 2,999.36
Rate: 6,741.52
Amount: 20,220,245.43

Descripción: Carrier Taxes for Transport | 522 Gbps Wavelength Diverse between Hillsboro, NV 98765
Cantidad: 4,621.34
Rate: 2,414.27
Amount: 11,157,162.52

Descripción: Installation of Cross Connect | 272 Gbps Fiber to tvkSG1
Cantidad: 7,312.35
Rate: 1,602.87
Amount: 11,720,746.44

Descripc

# try 3

In [10]:
import re

# Step 1: Improve the line merging logic
def merge_invoice_lines(ocr_text):
    lines = ocr_text.split("\n")
    merged_lines = []
    current_item = ""
    
    for line in lines:
        # Check if line contains tab-separated numbers (likely a complete line)
        if re.search(r"\t[\d,-]+\.\d+\t[\d,-]+\.\d+\t[\d,-]+\.\d+", line):
            if current_item:
                merged_lines.append(current_item)
            current_item = line
        elif line.strip() and not line.startswith("Invoice") and not line.startswith("Please"):
            if current_item:
                current_item += " " + line.strip()
            else:
                current_item = line.strip()
    
    if current_item:
        merged_lines.append(current_item)
    
    return merged_lines

# Step 2: Extract item details with improved pattern
def extract_items(merged_lines):
    pattern = re.compile(r"^(.*?)\t+([-\d,]+\.\d+)\t+([-\d,]+\.\d+)\t+([-\d,]+\.\d+)$")
    items = []
    
    for line in merged_lines:
        match = pattern.match(line)
        if match:
            items.append(match.groups())
    
    return items

# Process the invoice
merged_lines = merge_invoice_lines(ocr_text_2)
items = extract_items(merged_lines)

# Print results
print("Items Vendidos:")
for item in items:
    description, quantity, rate, amount = item
    print(f"Descripción: {description.strip()}")
    print(f"Cantidad: {quantity}")
    print(f"Rate: {rate}")
    print(f"Amount: {amount}\n")

Items Vendidos:
Descripción: Special Partnership Discount (03/2023)
Cantidad: 8.00
Rate: -561.00
Amount: -4,488.00

Descripción: Item Discount: Special Partnership Discount (11/2023)
Cantidad: 8.00
Rate: -296.00
Amount: -2,368.00

Descripción: Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)
Cantidad: 5,607.34
Rate: 656.07
Amount: 3,678,807.55



In [39]:
import re
import pandas as pd
import numpy as np
# Texto extraído del OCR
ocr_text_2 = """
Invoice\nswitch\tDallas, TX 75267-4592\nPO Box 674592\t\t\t\t\t\t\t\tPage 1 of 1\n\n\tInvoice Date Due Date\tInvoice No.\n\t09/22/23\t08/27/24\t1556267\n\nToni Hackel\n6600 Kalanianaole Hwy Ste 200,\nHonolulu\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of March\nU-08805490\t\t\t\tPO-2OPC-4316\nDescription\t\t\t\tQuantity\tRate\tAmount\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCarrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\nand Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\nTransport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21\n\n\tTotal USD\t$58,164,641.64\n\nPlease update your system with our new remittance instruction located on this invoice. Please contact accounts receivable@switch.com with any\nquestions.\n\nPlease make payments to: Switch, Ltd.\nWire/ACH Payment: PNC Bank, N.A.\tAccount No.:3802512896 ACH Routing:083009060 Wire Routing: 043000096 SWIFT:PNCCUS33\nFor questions, please call or email Accounts Receivable Department.\nPhone No.\tFax No.\t\t\tE-Mail\t\t\t\t\tWeb Site\n702.522.2940\t702.444.9546\taccountsreceivable@switch.com\t\t\twww.switch.com\nPlease remit payment for amount due. If payment submitted is different from amount due, a detailed statement of where to apply the funds must be included\nwith payment."""

# Paso 1: Identificar la sección de precios
start_pattern = r"Description\t+\t+Quantity\t+Rate\t+Amount\n"
end_pattern = r"\n\n\tTotal"

match_start = re.search(start_pattern, ocr_text_2)
match_end = re.search(end_pattern, ocr_text_2)

if match_start and match_end:
    price_section = ocr_text_2[match_start.end():match_end.start()]
else:
    raise ValueError("No se pudo identificar la sección de lista de precios.")

# data = price_section
# # Procesar los datos en líneas
# # Separar líneas
# import pandas as pd
# import numpy as np

# # Define the data
# data = '''Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCarrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\nand Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\nTransport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21'''

# # Split the data into lines
# lines = data.splitlines()

# # Initialize empty lists for the data
# types = []
# descriptions = []
# unit_costs = []
# quantities = []
# totals =[]

# # Iterate through the lines and extract the data
# for line in lines:
#     parts = line.split('\t')
#     # Check if the line contains the expected number of fields
#     if len(parts) == 4:
#         # If yes, extract the data
#         type_desc, unit_cost, quantity, total = parts
#         try:
#             # Try to split the type and description
#             type, desc = type_desc.split(' | ', 1)
#         except ValueError:
#             # If the split fails, assign the whole string to description and set type to None
#             type = None
#             desc = type_desc
#         # Append the extracted data to the lists
#         types.append(type)
#         descriptions.append(desc)
#         unit_costs.append(unit_cost)
#         quantities.append(quantity)
#         totals.append(total)
#     else:
#         # If the line does not contain the expected number of fields, assign None to all fields
#         types.append(None)
#         descriptions.append(None)
#         unit_costs.append(None)
#         quantities.append(None)
#         totals.append(None)

# # Create the DataFrame
# df = pd.DataFrame({
#     'Type': types,
#     'Description': descriptions,
#     'Unit Cost': unit_costs,
#     'Quantity': quantities,
#     'Total': totals
# })

# # Display the first 5 rows
# df

# Paso 2: Unir líneas partidas y agregar líneas sin precio a la anterior
ocr_lines = price_section.split("\n")
merged_lines = []
buffer = ""

for line in ocr_lines:
    if re.search(r"\t\d{1,3}(,\d{3})*\.\d{2}\t\d{1,3}(,\d{3})*\.\d{2}\t\d{1,3}(,\d{3})*\.\d{2}", line):
        if buffer:
            merged_lines.append(buffer.strip())
        buffer = line
    elif re.search(r"\(\d{2}/\d{4} Taxes\)", line):  # Línea sin precios
        buffer += " " + line.strip()
    else:
        buffer += " " + line.strip()

if buffer:
    merged_lines.append(buffer.strip())

# Paso 3: Extraer los datos con regex
pattern = re.compile(r"(.+?)\t+([\d,]+\.?\d*)\t+([\d,]+\.?\d*)\t+([\d,]+\.?\d*)")
items = pattern.findall("\n".join(merged_lines))

# Imprimir resultados
print("Items Vendidos:")
for item in items:
    description, quantity, rate, amount = item
    print(f"Descripción: {description.strip()}\nCantidad: {quantity}\nRate: {rate}\nAmount: {amount}\n")

Items Vendidos:
Descripción: Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber
Cantidad: 2,912.98
Rate: 934.09
Amount: 2,720,985.49

Descripción: Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and
Cantidad: 3,500.87
Rate: 6,229.33
Amount: 21,808,074.52

Descripción: Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)
Cantidad: 5,607.34
Rate: 656.07
Amount: 3,678,807.55

Descripción: Carrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765
Cantidad: 3,949.46
Rate: 7,128.95
Amount: 28,155,502.87

Descripción: Transport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)
Cantidad: 942.71
Rate: 1,918.01
Amount: 1,808,127.21



In [32]:
data

'Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCarrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\nand Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\nTransport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21'

# deepseek try

In [11]:
import re

# Texto extraído del OCR
ocr_text_2 = """ Invoice\nswitch\tDallas, TX 75267-4592\nPO Box 674592\t\t\t\t\t\t\t\tPage 1 of 1\n\n\tInvoice Date Due Date\tInvoice No.\n\t09/22/23\t08/27/24\t1556267\n\nToni Hackel\n6600 Kalanianaole Hwy Ste 200,\nHonolulu\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of March\nU-08805490\t\t\t\tPO-2OPC-4316\nDescription\t\t\t\tQuantity\tRate\tAmount\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCarrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\nand Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\nTransport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21\n\n\tTotal USD\t$58,164,641.64\n\nPlease update your system with our new remittance instruction located on this invoice. Please contact accounts receivable@switch.com with any\nquestions.\n\nPlease make payments to: Switch, Ltd.\nWire/ACH Payment: PNC Bank, N.A.\tAccount No.:3802512896 ACH Routing:083009060 Wire Routing: 043000096 SWIFT:PNCCUS33\nFor questions, please call or email Accounts Receivable Department.\nPhone No.\tFax No.\t\t\tE-Mail\t\t\t\t\tWeb Site\n702.522.2940\t702.444.9546\taccountsreceivable@switch.com\t\t\twww.switch.com\nPlease remit payment for amount due. If payment submitted is different from amount due, a detailed statement of where to apply the funds must be included\nwith payment."""

# Paso 1: Unir líneas partidas
ocr_lines = ocr_text_2.split("\n")
merged_lines = []
buffer = ""

for line in ocr_lines:
    # Si la línea tiene números (potencialmente una línea de datos), la añadimos al buffer
    if re.search(r"\t\d{1,3}(,\d{3})*.\d{2}\t", line):
        if buffer:
            merged_lines.append(buffer.strip())
        buffer = line
    else:
        # Si no, la añadimos al buffer como parte de la descripción
        buffer += " " + line.strip()

# Asegurarse de añadir el último buffer si existe
if buffer:
    merged_lines.append(buffer.strip())

# Paso 2: Extraer los datos con regex
pattern = re.compile(r"(.+?)\t+([\d,.-]+)\t+([\d,.-]+)\t+([\d,.-]+)")
items = pattern.findall("\n".join(merged_lines))

# Imprimir resultados
print("Items Vendidos:")
for item in items:
    description, quantity, rate, amount = item
    print(f"Descripción: {description.strip()}\nCantidad: {quantity}\nRate: {rate}\nAmount: {amount}\n")


Items Vendidos:
Descripción: Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber
Cantidad: 2,912.98
Rate: 934.09
Amount: 2,720,985.49

Descripción: Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and
Cantidad: 3,500.87
Rate: 6,229.33
Amount: 21,808,074.52

Descripción: Special Partnership Discount (03/2023)
Cantidad: 8.00
Rate: -561.00
Amount: -4,488.00

Descripción: Item Discount: Special Partnership Discount (11/2023)
Cantidad: 8.00
Rate: -296.00
Amount: -2,368.00

Descripción: Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)
Cantidad: 5,607.34
Rate: 656.07
Amount: 3,678,807.55

Descripción: Carrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765
Cantidad: 3,949.46
Rate: 7,128.95
Amount: 28,155,502.87

Descripción: Transport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)
Cantidad: 942.71
Rate: 1,918.01
Amount: 1,808,127.21

